In [161]:
import numpy as np
from scipy.optimize import nnls

Our goal is to optimize the function $||AB - X||_F^2 + \lambda||Av-y||^2$ where $A,B$ are constrained to be non-negative while $v$ is not.

In [162]:
def matrix_nnls(A,X):
    
    '''Returns the non-negative matrix B which minimizes ||AB - X||^2'''
    
    m,k = A.shape
    m_,n = X.shape
    
    if m != m_:
        raise ValueError('First dimensions on inputs must match')
    
    #display("X:", X)
    #display("m,n:, ", m, n)
    
    
    X_cols = [X[:,i] for i in range(0,n)]
    nnls_by_col = [nnls(A, col)[0] for col in X_cols]
    return np.column_stack(nnls_by_col)

In [163]:
def do_training(X, y_vec, lmda, k, num_iters=100):
    
    '''Iteratively minimizes $||AB - X||_F^2 + \lambda||Av-y||^2$ where X and y are given, with A having k columns and B having
    k rows. A and B are constrained to be non-negative.  v is not.'''
        
    m,n = X.shape
    y = y_vec.reshape((m,1))

    
    #random initializations
    A = np.random.normal(size=(m,k))
    B = np.random.normal(size=(k,n))
    v = np.random.normal(size=(k,1))
    
    for i in range(0,num_iters):
        B = matrix_nnls(A,X)
        v = np.linalg.lstsq(A,y,rcond=None)[0]
        
        #display("B", B)
        #display("v", v)
        
        A = np.transpose(matrix_nnls( 
            np.transpose(np.concatenate( (B, lmda*v),axis=1) ), 
            np.transpose(np.concatenate( (X, lmda*y),axis=1) )))
    
    return A,B,v

Example:

In [165]:
X = np.array([[1,2,3],[4,5,6],[7,8,9]])
y = np.array([2,1,1])

In [179]:
do_training(X,y,lmda=1, k=2, num_iters=100)

(array([[0.00536359, 1.14179366],
        [1.3913214 , 2.22068166],
        [2.51295013, 3.41255438]]),
 array([[1.62150018e+00, 8.10750090e-01, 1.41707768e-14],
        [8.38182643e-01, 1.74629329e+00, 2.65440394e+00]]),
 array([[-1.98085501],
        [ 1.73573946]]))